In [ ]:
!pip install langchain --quiet
!pip install langchain_community --quiet
!pip install langchain_huggingface --quiet
!pip install unstructured --quiet
!pip install nltk --quiet
!pip install huggingface_hub --quiet

In [ ]:

from langchain_community.document_loaders import TextLoader

loader = TextLoader('resumes/sample_resume.txt')
docs = loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embedding=HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5')
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
vectorstore=InMemoryVectorStore.from_documents(splits,embedding)
retriever=vectorstore.as_retriever()

In [ ]:
from google.colab import userdata
huggingface_api_key=userdata.get('HUGGINGFACE_API_KEY')

In [ ]:
from langchain_community.chat_models import ChatHuggingFace
from langchain_community.llms import HuggingFaceHub

llm=HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token=huggingface_api_key,
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        # "top_k": 5,
        # "temperature": 0.3,
        # # "repetition_penalty": 1.00,
    },

)

In [ ]:
import huggingface_hub
huggingface_hub.login(huggingface_api_key)
chat_model=ChatHuggingFace(llm=llm)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
jd="""Job Title: Data Engineer

Department: Engineering/Data

Location: [City, State]

Reports To: [Manager Title, e.g., Director of Engineering]

About [Company Name]

[Insert a brief and engaging description of your company, its mission, and values.]

Job Summary

We are seeking a talented and passionate Data Engineer to join our growing team. In this role, you will be responsible for designing, building, and maintaining scalable data pipelines and infrastructure to support our data-driven initiatives. You will work closely with data scientists, engineers, and product managers to deliver innovative solutions that drive business value.

Responsibilities

Design, build, and maintain scalable data pipelines for ingesting, processing, and transforming large volumes of data from various sources.
Develop and implement data quality checks and monitoring processes to ensure data accuracy and reliability.
Collaborate with data scientists and analysts to understand data requirements and provide data solutions.
Work with cloud-based data platforms and technologies (e.g., AWS, Google Cloud, Azure).
Optimize data storage and retrieval mechanisms for performance and scalability.
Implement data security and access controls to protect sensitive information.
Stay up-to-date with the latest advancements in data engineering and big data technologies.
Qualifications

Bachelor's or Master's degree in Computer Science, Engineering, or a related field.
3+ years of experience in data engineering or a related role.
Strong programming skills in Python or Java.
Experience with data warehousing and ETL (Extract, Transform, Load) processes.
Experience with big data technologies such as Hadoop, Spark, or Hive.
Familiarity with data modeling and database design principles.
Excellent communication and teamwork skills.
Bonus Points

Experience with cloud-based data platforms (e.g., AWS, Google Cloud, Azure).
Experience with data visualization tools (e.g., Tableau, Power BI).
Experience with machine learning or data science concepts.
Contributions to open-source projects or data engineering communities.
Benefits

[List the benefits your company offers, such as competitive salary, health insurance, paid time off, and professional development opportunities.]

To Apply

Please submit your resume and cover letter to [email protected]

We are an equal opportunity employer and value diversity at our company. We do not discriminate on the basis of race, religion, color, national origin, gender, sexual orientation, age, marital status, veteran status, or disability status."""

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage,AIMessage

system_prompt=(
    "You are an assistant for resume analysing tasks for HR department. "
    "Use the following pieces of retrieved context and job description to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

qa_chain=create_stuff_documents_chain(chat_model,qa_prompt)
rag_chain=create_retrieval_chain(retriever,qa_chain)

In [ ]:
skill_query = f"What are the key skills of the cadidate?"
skill_response=rag_chain.invoke(
    {"input":skill_query}
)

suitability_query = f"Based on the available data of the cadidate from {skill_response} and job description which the candidate is applying for: {jd}, is the candidate suitable for the position mentioned in the job description or best suited for any other positions? Explain your reasoning."

suitability_response=rag_chain.invoke(
    {"input":suitability_query}
)


In [ ]:
print(str(skill_response['answer']).split("|>")[-1])


The key skills of the candidate, as listed in their resume, are:

1. Highly motivated and skilled Machine Learning Engineer with 3+ years of experience in developing and implementing machine learning models.
2. Proven ability to solve complex business problems using advanced algorithms and techniques.
3. Strong programming skills in Python and expertise in popular machine learning libraries such as scikit-learn, TensorFlow, and PyTorch.
4. Passionate about staying up-to-date with the latest advancements in AI and contributing to innovative projects.
5. Data analysis skills using Pandas, NumPy, and SQL.
6. Cloud computing skills using AWS and Azure.
7. Version control skills using Git.
8. Data visualization and communication skills.
9. Teamwork skills.

These skills demonstrate the candidate's proficiency in machine learning, programming, data analysis, cloud computing, and collaboration, making them a strong candidate for machine learning and AI-related roles.


In [ ]:
print(str(suitability_response['answer']).split('|>')[-1])


Based on the candidate's skills and experience listed in their resume, they appear to be a strong fit for the Data Engineer position described in the job description. Their expertise in designing, building, and maintaining scalable data pipelines, implementing data quality checks, and working with cloud-based data platforms aligns well with the responsibilities outlined in the job description. Additionally, their experience in data modeling and database design principles, as well as their strong programming skills in Python and Java, further support their qualifications for the role. While the candidate's experience in machine learning and data science concepts is a bonus, it is not a requirement for the Data Engineer position, as the focus of the role is primarily on designing and implementing data pipelines and infrastructure. Overall, the candidate's skills and experience make them a strong candidate for the Data Engineer position at the company.
